In [1]:
import sys
sys.path.append("..")
from src.environment import *
from src.elements import *
from src.dynamics import *
from utils.test_utils import *
from src.methods import *

In [5]:
dynamics = D2Q9(tau =.1,delta_t = .1)
key = Key(1234)
Element.dynamics = dynamics
Environment.dynamics = dynamics
n_cells = 10
n_faces = 10
n_nodes = 10
cells = [Cell.pdf_init(jax.random.uniform(key(),dynamics.ones_pdf().shape),jax.random.randint(key(),(4,),0,n_faces),jax.random.randint(key(),(4,),-1,1)) for i in range(n_cells)]
faces = [Face.pdf_init(jax.random.uniform(key(),dynamics.ones_pdf().shape),jnp.asarray([jax.random.randint(key(),(2,),0,2),jax.random.randint(key(),(2,),0,n_nodes)]),jax.random.randint(key(),(2,),0,n_cells),jax.random.normal(key(),(2,))+.1,jax.random.normal(key(),(2,)),jax.random.normal(key(),())) for i in range(n_faces)]
nodes = [Node.pdf_init(jax.random.uniform(key(),dynamics.ones_pdf().shape),jax.random.randint(key(),(),0,2),jnp.asarray([]),jnp.asarray([])) for i in range(100)]
for node in nodes:
    n = jax.random.randint(key(),(),2,10)
    node.type = jnp.asarray([1])
    node.cell_dists = jnp.abs(jax.random.normal(key(),(n,)))
    node.cells_index = jax.random.randint(key(),(n,),0,n_cells)
env = Environment.create(cells,faces,nodes)
env.setMethods(Methods)

In [6]:
params,config = env.init()
print(jax.tree.structure(params))
print(jax.tree.structure(config))

PyTreeDef({'cells': {'pdf': *, 'pdf_eq': *, 'rho': *, 'vel': *}, 'faces': {'flux': *, 'pdf': *, 'rho': *, 'vel': *}, 'nodes': {'Internal': {'pdf': *, 'rho': *, 'vel': *}, 'Pressure': {'pdf': *, 'rho': *, 'vel': *}, 'Velocity': {'pdf': *, 'rho': *, 'vel': *}}})
PyTreeDef({'cells': {'faces_index': *, 'faces_n': *}, 'faces': {'L': *, 'n': *, 'nodes_index': *, 'stencil_cells_index': *, 'stencil_dists': *}, 'nodes': {'Internal': {'cell_dists': *, 'cells_index': *, 'type': *}, 'Pressure': {'cell_dists': *, 'cells_index': *, 'type': *}, 'Velocity': {'cell_dists': *, 'cells_index': *, 'type': *}}})


In [ ]:
jnp.where(dynamics.KSI[:,1] == 1)

In [ ]:
env.methods.test_vmap(env.methods.test_vmap(params,config),config)["cells"]["pdf"].shape

Cell Eq Test

In [ ]:
jitted = jax.jit(env.methods.calc_cell_eqs)
jitted(params)
%timeit -n100 -r10 env.methods.calc_cell_eqs(params)
%timeit -n100 -r10 jitted(params)

In [ ]:
print(params["nodes"]["Velocity"]["pdf"].shape)

(10, 10, 10, 100, 9)


Cell Macros Test

In [7]:
params = env.methods.calc_cell_macros(params,config)
params = env.methods.calc_cell_eqs(params,config)
params = env.methods.calc_node_pdfs(params,config)
params = env.methods.calc_face_pdfs(params,config)
params = env.methods.calc_cell_pdfs(params,config)

In [8]:
params1 = env.step(params,config)

In [9]:
jitted = jax.jit(env.methods.calc_cell_macros)
jitted(params,config)
%timeit -n100 -r10 env.methods.calc_cell_macros(params,config)
%timeit -n100 -r10 jax.block_until_ready(jitted(params,config))

325 μs ± 9.53 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
28.6 μs ± 904 ns per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [10]:
jitted = jax.jit(env.methods.calc_cell_eqs)
jitted(params,config)
%timeit -n100 -r10 env.methods.calc_cell_eqs(params,config)
%timeit -n100 -r10 jax.block_until_ready(jitted(params,config))

1.13 ms ± 27.1 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
32.6 μs ± 7.28 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [11]:
jitted = jax.jit(env.methods.calc_node_pdfs)
jitted(params,config)
%timeit -n100 -r10 env.methods.calc_node_pdfs(params,config)
%timeit -n100 -r10 jax.block_until_ready(jitted(params,config))

5.54 ms ± 209 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
37.6 μs ± 1.54 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [12]:
jitted = jax.jit(env.methods.calc_face_pdfs)
jitted(params,config)
%timeit -n100 -r10 env.methods.calc_face_pdfs(params,config)
%timeit -n100 -r10 jax.block_until_ready(jitted(params,config))

18.9 ms ± 592 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
35.8 μs ± 2.41 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [13]:
jitted = jax.jit(env.methods.calc_cell_pdfs)
jitted(params,config)
%timeit -n100 -r10 env.methods.calc_cell_pdfs(params,config)
%timeit -n100 -r10 jax.block_until_ready(jitted(params,config))

1.32 ms ± 20.6 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
33.8 μs ± 2.18 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [14]:
jitted = jax.jit(env.step)
jitted(params,config)
%timeit -n10 -r10 env.step(params,config)
%timeit -n10 -r10 jax.block_until_ready(jitted(params,config))

27.3 ms ± 859 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)
43.6 μs ± 5.5 μs per loop (mean ± std. dev. of 10 runs, 10 loops each)
